# Web Application for an ETF Analyzer

In this Project, we build a financial database and web application using SQL, Python, and the Voilà library to analyze the performance of a hypothetical FinTech ETF. This notebook is used for the analysis of a fintech ETF that consists of four stocks: GDOT, GS, PYPL, and SQ. Each stock has its own table in the `etf.db` database. 
(GDOT: Green Dot Corporation, GS: Goldman Sacks, PYPL: Paypal Holdings, SQ: Block)

We analyze the daily returns of the ETF stocks both individually and as a whole. Then deploy the visualizations to a web application by using the Voilà library. The detailed instructions are divided into the following parts:

* Analyze a single asset in the ETF

* Optimize data access with Advanced SQL queries

* Analyze the ETF portfolio

* Create a new database where to store the data of the ETF returns equally weighted.The idea is to not to alter the original.

* Deploy the notebook as a web application


## Imports of the required libraries, initiation of the SQLite database, population of the database with records from the `etf.db` seed file that is included in the repository, creates the database engine, and confirms that data tables that it now contains.

In [ ]:
# Importing the required libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
import sqlalchemy
import datetime
from datetime import date, datetime
from sqlalchemy import inspect

# Create a temporary SQLite database and populate the database with content from the etf.db seed file
database_connection_string = 'sqlite:///etf.db'

# Create an engine to interact with the SQLite database
engine = sqlalchemy.create_engine(database_connection_string)

# Confirm that table names contained in the SQLite database.
print("Tables names for data from stocks Green Dot Inc, Goldman Scahs Group Inc, Paypal Inc, and Square Inc.")
inspect(engine).get_table_names()


In [ ]:
#sqlalchemy.create_engine?

## Analyze a single asset in the FinTech ETF


### Step 1: Write a SQL `SELECT` statement by using an f-string that reads all the PYPL data from the database. Using the SQL `SELECT` statement, we execute a query that reads the PYPL data from the database into a Pandas DataFrame.

In [ ]:
# Write a SQL query to SELECT all of the data from the PYPL table
query = """
SELECT * from PYPL
"""

# Use the query to read the PYPL data into a Pandas DataFrame and set index to "time"
fmt='%Y%m%d %H:%M:%S'
pypl_dataframe = pd.read_sql_query(query, con=engine, parse_dates = {'time':fmt} )
pypl_dataframe = pypl_dataframe.set_index('time')

print("\033[1m  Table with Paypal Inc. prices, volume and daily returns")
pypl_dataframe

### Step 2: Use the `head` and `tail` functions to review the first five and the last five rows of the DataFrame. We save the beginning and end dates that are available from this dataset, since we’ll use this information to complete the analysis.

In [ ]:
# Print Beggining and End date of the Period.
print(f"Beggining of period: {pypl_dataframe.index[0]}")
print(f"End of period      : {pypl_dataframe.index[-1]}")

#Calculate lenght of period to calculate actual annualized return later
period = (pypl_dataframe.index[-1] - pypl_dataframe.index[0])
period_in_years = period.days/365.25
print(f"Period in days is: {period.days}, which are {period_in_years:,.2f} years \n\n")

# View the first 5 rows of the DataFrame.
print("\033[1m  Firsts and lasts columns of the pypl_dataframe, with data of Paypal stock. \n")
display(pypl_dataframe.head())
display(pypl_dataframe.tail())

In [ ]:
pypl_dataframe["Daily Returns %"] = pypl_dataframe['daily_returns']*100

### Step 3: Interactive visualization for the PYPL daily returns using hvPlot.

In [ ]:
# Create an interactive visualization with hvplot to plot the daily returns for PYPL.
pypl_dataframe.hvplot(
    title="PYPL Daily Returns (%)"
    ,y='Daily Returns %'
    ,xlabel= 'Date'
    ,ylabel='Returns (%)'
    ,width=800
).opts(
    color='blue')

### Step 4: Interactive visualization for the PYPL cumulative returns. 

In [ ]:
# Representing the cummulative investment
growth_of_1usd_investment=(1+pypl_dataframe["daily_returns"]).cumprod()

#Transforming a series to a dataframe and renaming columns
growth_of_1usd_investment = growth_of_1usd_investment.to_frame().rename(columns={'daily_returns':'Growth of 1 USD Investment'})
print("\033[1m Table: Evolution of a $1 initial investment on Dec 15th 2016 on the ETF.")

display(growth_of_1usd_investment)

# Create an interactive visaulization with hvplot to plot the cumulative returns for PYPL.
growth_of_1usd_investment.hvplot(
    title="Paypal Holdings Inc -- Growth of 1 USD Initial Investment -- Period Dec-16-2016 to Dec 4th 2020"
    ,ylabel="Initial Investment \n plus Cumulative Return"
    ,xlabel= "Date"
    ,width=900

)

## Optimize the SQL Queries

For this part, we continue to analyze a single asset (PYPL) from the ETF. We use SQL queries to optimize the efficiency of accessing data from the database.



### Step 1: Access the closing prices for PYPL that are greater than 200 by completing the following steps:

    - Select the dates where the PYPL closing price was higher than 200.0.

    - Read the data from the database into a Pandas DataFrame, and then review the resulting DataFrame.

    - Select the “time” and “close” columns for those dates where the closing price was higher than 200.0.



In [ ]:
# Write a SQL SELECT statement to select the time column 
# where the PYPL closing price was higher than 200.0.
query = """
SELECT time 
FROM PYPL
WHERE close > 200
"""

# Using the query, read the data from the database into a Pandas DataFrame, and convert date strings to date
fmt = '%Y%m%d %H:%M:%S'
pypl_dates_higher_than_200 = pd.read_sql_query(query, engine, parse_dates={'time':fmt})

# Review the resulting DataFrame
print("\033[1m Older dates when close price of Paypal is higher than $200. Data comes from SQL database:")
display(pypl_dates_higher_than_200.head())

# Select those dates from the pypl dataset
pypl_higher_than_200 = pypl_dataframe.loc[pypl_dates_higher_than_200['time'],'close'].to_frame()
print("\033[1m  Older dates and close price of Paypal when higher than $200 in pandas dataframe")
display(pypl_higher_than_200.head())

### Step 2: Find the top 10 daily returns for PYPL by completing the following steps:

    -  Write a SQL statement to find the top 10 PYPL daily returns. For this purpose we:

        * Select only the “time” and “daily_returns” columns.

        * Use `ORDER` to sort the results in descending order by the “daily_returns” column.

        * Use `LIMIT` to limit the results to the top 10 daily return values.

    - Using the SQL statement, read the data from the database into a Pandas DataFrame, and then review the resulting DataFrame.


In [ ]:
# Write a SQL SELECT statement to select the time and daily_returns columns
# Sort the results in descending order and return only the top 10 return values
query = """
SELECT time, daily_returns
FROM PYPL
ORDER by daily_returns  desc
LIMIT 10
"""

# Using the query, read the data from the database into a Pandas DataFrame
# Counting is useful to visualize amount of data, so index is not change to time
fmt = '%Y%m%d %H:%M:%S'
pypl_top_10_returns = pd.read_sql_query(query, engine, parse_dates={'time':fmt})

pypl_top_10_returns['daily_returns']=pypl_top_10_returns['daily_returns']*100

print("\033[1m Table with the dates when the top 10 larger daily returns of Paypal Inc. occured:")
display(pypl_top_10_returns[['time']])
    
# Review the resulting DataFrame
print ("\n")
print("\033[1m Table with the top 10 larger daily returns of Paypal stock in percentages (%):")
display(round(pypl_top_10_returns,2))


## Analyze the Fintech ETF Portfolio

For this part, we build the entire ETF portfolio and then evaluate its performance. To do so, we build the ETF portfolio by using SQL joins to combine all the data for each asset.


### Step 1: Write a SQL query to join each table in the portfolio into a single DataFrame. To do so, we complete the following steps:

    - Use a SQL inner join to join each table on the “time” column. Access the “time” column in the `GDOT` table via the `GDOT.time` syntax. Access the “time” columns from the other tables via similar syntax.

    - Using the SQL query, read the data from the database into a Pandas DataFrame. 
    
    - Finally, we review the resulting DataFrame.

In [ ]:
# Here we create a SQL query to join each table in the portfolio into a single DataFrame 
# We use the time column from each table as the basis for the join
query = """
SELECT  *
FROM GDOT, GS, PYPL, SQ
WHERE GDOT.time = GS.time
AND   PYPL.time = SQ.time
AND   GDOT.time = PYPL.time
"""
#['GDOT', 'GS', 'PYPL', 'SQ']
# Using the query, read the data from the database into a Pandas DataFrame
frm='%Y%m%d %H:%M:%S'
etf_portfolio = pd.read_sql_query(query, engine, parse_dates={'time':frm})

# Review the resulting DataFrame
print('\n')
print("\033[1m Join Tables from ['GDOT', 'GS', 'PYPL', 'SQ'] on dates")
display(etf_portfolio.head())

### Step 2: Create a DataFrame that averages the “daily_returns” columns for all four assets. Review the resulting DataFrame.

 We assume that this ETF contains equally weighted returns, and average the daily returns for all assets to get the average returns of the portfolio. We use the average returns of the portfolio to calculate the annualized returns and the cumulative returns. For the calculation to get the average daily returns for the portfolio, we use the following code:

 ```python
 etf_portfolio_returns = etf_portfolio['daily_returns'].mean(axis=1)
 ```

In [ ]:
# Create a DataFrame that averages the “daily_returns” columns for all four assets. Review the resulting DataFrame.
etf_portfolio_returns = etf_portfolio['daily_returns'].mean(axis=1)
print('\033[1mETF Portfolio Returns (%)')
display(round((etf_portfolio_returns*100),2).head(10))

In [ ]:
# As a second view, we create a DataFrame that displays the value of the “daily_returns” for all four assets only, and assign an index.
# Use the time column from each table as the basis for the join
query2 = """
SELECT  GDOT.time, GDOT.daily_returns as 'GDOT.daily_returns',
     GS.daily_returns as 'GS.daily_returns',
   PYPL.daily_returns as 'PYPL.daily_returns',
     SQ.daily_returns as 'SQ.daily_returns'
FROM GDOT, GS, PYPL, SQ
WHERE GDOT.time = GS.time
AND   PYPL.time = SQ.time
AND   GDOT.time = PYPL.time
"""

#['GDOT', 'GS', 'PYPL', 'SQ']
# Using the query, read the data from the database into a Pandas DataFrame
fmt='%Y%m%d %H:%M:%S'
etf_portfolio2 = pd.read_sql_query(
        query2
        , engine
        , parse_dates={'time':fmt}
)
etf_portfolio2 = etf_portfolio2.set_index("time") 

In [ ]:
print('\n')
print('\033[1m                         Daily individual returns in time (%)')
display(round((etf_portfolio2 * 100),2))


# We repeat the calculation of average daily returns using this table
etf_portfolio_returns = etf_portfolio2.mean( axis=1 )

# Review the resulting DataFrame
display("ETF Returns (%)")
display(round(etf_portfolio_returns*100,2))

### Step 3: Use the average daily returns in the etf_portfolio_returns DataFrame to calculate the annualized returns for the portfolio. Display the annualized return value of the ETF portfolio.

To calculate the expected annualized returns, we multiply the mean of the `etf_portfolio_returns` values by 252.

To convert the decimal values to percentages, we multiply the results by 100 before printing or plotting the values.

In [ ]:
# Use the average daily returns in the etf_portfolio_returns DataFrame 
# to calculate the annualized return for the portfolio. 
annualized_etf_portfolio_returns = etf_portfolio_returns.mean()*252

print(f"The expected annualized return, calculated using daily average return in the period, times 252 trading days is: {annualized_etf_portfolio_returns*100:,.2f}% ")



### Step 4: We use the average daily returns in the `etf_portfolio_returns` DataFrame to calculate the cumulative returns of the ETF portfolio.


In [ ]:
# Use the average daily returns provided by the etf_portfolio_returns DataFrame 
# to calculate the cumulative returns
# This is the growth of 1[USD] initial investment
etf_cumulative_returns = (1 + etf_portfolio_returns).cumprod()

In [ ]:
# ROI
etf_cumulative_return_above_initial_investment = etf_cumulative_returns[len(etf_cumulative_returns) - 1] - 1
growth_of_1usd_initial_investment = etf_cumulative_return_above_initial_investment + 1

# Display the final cumulative return value
print(f"The cumulative return of the investment in the full period, above the initial investment (no-annualized) is of {etf_cumulative_return_above_initial_investment*100:,.2f}%")
print(f"The growth of $1.00 initial investment in the full period is ${growth_of_1usd_initial_investment:.2f} ")

In [ ]:
# Adjusting columns names for proper graph variables
etf_cumulative_returns_df = pd.DataFrame(etf_cumulative_returns, columns=['Growth of 1[USD] Initial Investment'])

etf_cumulative_returns_df.tail()

### Step 5: Using hvPlot, we create an interactive line plot that visualizes the cumulative return values of the ETF portfolio. Reflect the “time” column of the DataFrame on the x-axis. 

In [ ]:
# Using hvplot, create an interactive line plot that visualizes the ETF portfolios cumulative return values.
etf_cumulative_returns_df.hvplot(
    title="ETF - Equally Weighted FinTech Stocks (GDOT, GS, PYPL, SQ) Growth of 1 USD Initial Investment -- Dec-16-2016 to Dec 4th 2020"
    , ylabel="Cumulative Investment [$]"
    , xlabel= "Date"
    , width=900
)

### Step 6: Finally we will save the results in a new table in a new database, to keep the original etf.db file intact for future use of the script. This database will just contain just ETF returns. In this case, we will include the equally weighted column.

In [ ]:
# We will save the results on a new database called etf_returns that will contain only full etf returns, no components.
# Create a temporary SQLite database and save it as etf_returns.db
database_connection_string2 = 'sqlite:///etf_returns.db'

# Create an engine to interact with the SQLite database
engine2 = sqlalchemy.create_engine(database_connection_string2)
inspect(engine2).get_table_names()


In [ ]:
# We prepare column names for the data
etf_portfolio_returns_df = etf_portfolio_returns.to_frame()
etf_portfolio_returns_df.columns =['Equally_weighted']
etf_portfolio_returns_df

In [ ]:
# We insert in the new table the ETF equally weighted returns just calculated
etf_portfolio_returns_df.to_sql('ETF_returns', engine2, index=True, if_exists='replace')

In [ ]:
# We inspect the column names in the table to confirm is there 
columns_table = inspect(engine2).get_columns('ETF_returns') #schema is optional

# We print them with the datatype
for c in columns_table :
   print(c['name'], c['type'])

In [ ]:
# We check the data have been saved in etf_returns.db
read_data_query = """
    SELECT * from ETF_returns
"""
pd.read_sql_query(read_data_query, con=engine2)

#### Deployment of the Notebook as a Web Application

For this part, we completed the following steps:

1. Use the Voilà library to deploy the notebook as a web application locally on the computer.

2. Include a screen recording in the GitHub repository, as well as screenshots in the "README.md" file to show how the web application appears when using Voilà.

In [ ]:
# To include in future developments:
to use the INSERT statetment